In [1]:
!pip install tensorflow
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, LSTM,BatchNormalization,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras.regularizers import l2

# Lstm model after using encoder

In [2]:
X1=np.load("X11_25.npy")
X2=np.load("X22_25.npy")


In [3]:
X1.shape

(27672, 1600)

In [4]:
Y1=np.load("Y1_25.npy",allow_pickle=True)
Y2=np.load("Y2_25.npy",allow_pickle=True)

In [5]:
X1=np.float32(X1)
X2=np.float32(X2)
Y1=np.float32(Y1)
Y2=np.float32(Y2)

In [8]:
X1=np.reshape(X1,(X1.shape[0],25,64))
X2=np.reshape(X2,(X2.shape[0],25,64))

In [9]:
XT=np.concatenate((X1, X2), axis=0)

In [10]:
YT=np.concatenate((Y1, Y2), axis=0)

In [11]:
def createModel(inputShape,output):
# define model
    model = Sequential()
    model.add(LSTM(100, activation='relu',input_shape=inputShape,return_sequences=True))
    model.add(LSTM(100,activation='relu', return_sequences=False))
#     model.add(LSTM(64,activation='relu', return_sequences=False))
#     model.add(Dense(16, activation='linear'))
#     model.add(Dropout(0.3))
    model.add(Dense(output))

    opt = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='mae',metrics=['mae'],optimizer="adam")
    # model.compile(optimizer='adam', loss='mean_squared_error',metrics=['mse'])
    return model

In [12]:
time_steps = 25  # Number of time steps
feature_length = 64  # Length of each feature vector
input_shape = (time_steps, feature_length)

model=createModel(input_shape,50)

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 25, 100)           66000     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 50)                5050      
                                                                 
Total params: 151,450
Trainable params: 151,450
Non-trainable params: 0
_________________________________________________________________


In [14]:
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

In [15]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error,mean_squared_error
# load your data
X = XT  # your features
y = YT  # your target variable

# set up the k-fold cross-validation object
k =5   # number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# initialize a list to store the validation accuracies
val_mae = []
val_mse = []
val_mape=[]

# loop over the k folds
for train_index, val_index in kf.split(X):
    # split the data into training and validation sets
    X_train, y_train = X[train_index], y[train_index]
    X_val, y_val = X[val_index], y[val_index]

    # train your model on the training set
    model.fit(X_train, y_train)

    # evaluate the model on the validation set and store the mse
    y_pred = model.predict(X_val)
    val_acc = mean_absolute_error(y_val, y_pred)
    val_mae.append(val_acc)
    val_acc = mean_squared_error(y_val, y_pred)
    val_mse.append(val_acc)
    val_acc = MAPE(y_val, y_pred)
    val_mape.append(val_acc)
    

# print the mean and standard deviation of the validation mse
print(f'Validation mae: {np.mean(val_mae):.3f} ± {np.std(val_mae):.3f}')
print(f'Validation mse: {np.mean(val_mse):.3f} ± {np.std(val_mse):.3f}')
print(f'Validation mape: {np.mean(val_mape):.3f} ± {np.std(val_mape):.3f}')

304/304 [==============================] - 1s 4ms/step
Validation mae: 1.004 ± 0.334
Validation mse: 1.674 ± 0.990
Validation mape: 1.044 ± 0.345


In [16]:
loss, mae = model.evaluate(X2, Y2)
print('Test loss:', loss)
print('Test MAE:', mae)

655/655 [==============================] - 4s 5ms/step - loss: 0.9274 - mae: 0.9274
Test loss: 0.9274227023124695
Test MAE: 0.9274227023124695


In [17]:
model.save("lstm_model_withencoder_25.h5")

# CNN-LSTM model

In [19]:
# from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Flatten,Reshape

# Define CNN-LSTM model architecture
model2 = Sequential()
model2.add(Conv2D(32, (3,3), activation='relu', input_shape=(25,6,848)))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Flatten())
model2.add(Reshape((4,176)))
model2.add(LSTM(176, return_sequences=False))
model2.add(Dense(50, activation='linear'))

# Compile model
model2.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mae'])

# Print model summary
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 23, 4, 32)         244256    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 11, 2, 32)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 704)               0         
                                                                 
 reshape_1 (Reshape)         (None, 4, 176)            0         
                                                                 
 lstm_2 (LSTM)               (None, 176)               248512    
                                                                 
 dense_1 (Dense)             (None, 50)                8850      
                                                      

In [20]:
X11=np.load("X1_25.npy")
X22=np.load("X2_25.npy")

In [21]:
X11.shape,X22.shape

((27672, 25, 6, 848), (20929, 25, 6, 848))

In [22]:
X11=np.float32(X11)
X22=np.float32(X22)

In [23]:
Y1=np.float32(Y1)
Y2=np.float32(Y2)

In [24]:
Y1.shape,Y2.shape

((27672, 50), (20929, 50))

In [25]:
X11.shape

(27672, 25, 6, 848)

In [26]:
model2.fit(X11, Y1, epochs=10, verbose=1,batch_size=32, validation_split=0.2)

Epoch 1/10
692/692 [==============================] - 26s 36ms/step - loss: 46.3423 - mae: 46.3423 - val_loss: 2.5797 - val_mae: 2.5797
Epoch 2/10
692/692 [==============================] - 24s 35ms/step - loss: 2.6389 - mae: 2.6389 - val_loss: 2.4330 - val_mae: 2.4330
Epoch 3/10
692/692 [==============================] - 24s 35ms/step - loss: 1.4896 - mae: 1.4896 - val_loss: 1.1576 - val_mae: 1.1576
Epoch 4/10
692/692 [==============================] - 25s 37ms/step - loss: 0.7737 - mae: 0.7737 - val_loss: 0.9191 - val_mae: 0.9191
Epoch 5/10
692/692 [==============================] - 24s 34ms/step - loss: 0.5959 - mae: 0.5959 - val_loss: 0.8986 - val_mae: 0.8986
Epoch 6/10
692/692 [==============================] - 24s 35ms/step - loss: 0.4821 - mae: 0.4821 - val_loss: 0.6808 - val_mae: 0.6808
Epoch 7/10
692/692 [==============================] - 24s 35ms/step - loss: 0.3992 - mae: 0.3992 - val_loss: 0.6822 - val_mae: 0.6822
Epoch 8/10
692/692 [==============================] - 24s 35

In [ ]:
loss, mae = model2.evaluate(X22, Y2)
print('Test loss:', loss)
print('Test MAE:', mae)

375/655 [================>.............] - ETA: 2s - loss: 0.8507 - mae: 0.8507

In [24]:
model2.save("Cnn_lstm_model.h5")